In [1]:
import os
import re
import io
import sys
import time
import requests
import platform
from dotenv import load_dotenv
import subprocess
import numpy as np
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
EXAMPLE_PI = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(10_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [3]:
EXAMPLE_MATRIX = """
import time
import random

def generate_matrix(size):
    return [[random.random() for _ in range(size)] for _ in range(size)]

def matrix_multiply_without_numpy(size):
    matrix_a = generate_matrix(size)
    matrix_b = generate_matrix(size)
    
    result = [[0 for _ in range(size)] for _ in range(size)]
    
    start_time = time.time()
    
    # Matrix multiplication
    for i in range(size):
        for j in range(size):
            for k in range(size):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
    
    end_time = time.time()
    
    sum_value = sum(sum(row) for row in result)
    
    print(f"Matrix multiplication of {size}x{size} matrices (no NumPy)")
    print(f"Result sum: {sum_value:.6f}")
    print(f"Execution Time: {(end_time - start_time):.6f} seconds")

matrix_multiply_without_numpy(300)
"""

In [4]:
EXAMPLES = {
    "Pi Calculation": EXAMPLE_PI,
    "Matrix Multiplication": EXAMPLE_MATRIX
}

# Helper functions
def write_output(code, file_name):
    """Write the generated code to a file."""
    with open(file_name, "w") as f:
        f.write(code)

def system_message_for(language):
    """Create a system message tailored for the requested language."""
    return (
        f"You are an assistant that reimplements Python code in high-performance {language.upper()} for an M1 Mac. "
        f"Respond only with {language.upper()} code; do not explain your work other than occasional comments. "
        "Pay attention to number types to ensure no overflows and include all necessary packages.\n\n"
    )

def user_prompt_for(python):
    """Generate the user prompt."""
    return (
        "Rewrite this Python code in the requested language with the fastest possible implementation that produces "
        "identical output in the least time. Use appropriate syntax for the language.\n\n" + python
    )


In [5]:
def execute_python(code):
    """
    Execute Python code dynamically and capture its output.
    """
    output = io.StringIO()
    try:
        old_stdout = sys.stdout
        sys.stdout = output  # Redirect standard output to the StringIO object
        start_time = time.time()
        exec(code, {"np": np, "time": time})  # Execute code with minimal context
        end_time = time.time()
        execution_time = end_time - start_time
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        sys.stdout = old_stdout  # Restore standard output

    result = output.getvalue()
    if "Execution Time:" not in result:
        result += f"\nExecution Time: {execution_time:.6f} seconds"
    return result


In [6]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [7]:
def strip_code_fences(s: str) -> str:
    """
    Strip code fences and any thinking or extraneous text from the model output.
    Returns clean code only.
    """
    # First, try to extract code between fences if they exist
    code_pattern = re.compile(r"```(?:[a-zA-Z0-9_+-]*)\s*\n([\s\S]*?)\n```")
    code_match = code_pattern.search(s)
    
    if code_match:
        # If we found code between fences, return just that code
        return code_match.group(1).strip()
    
    # Otherwise, clean up the text more generally
    # Remove opening fences with optional language indicators
    s = re.sub(r"```(?:[a-zA-Z0-9_+-]*)\s*\n?", "", s)
    # Remove closing fences
    s = re.sub(r"\n?```\s*$", "", s)
    # Remove any thinking sections
    s = re.sub(r"(?i)thinking:|<thinking>[\s\S]*?</thinking>", "", s)
    # Remove common explanatory text that precedes or follows code
    s = re.sub(r"(?i)^.*?(?:here's|here is|the|implementation|code).*?\n", "", s)
    s = re.sub(r"(?i)\n.*?(?:this code|explanation|note:).*?$", "", s)
    
    return s.strip()

In [ ]:
def execute_cpp(code):
    """Compile and execute C++ code."""
    write_output(strip_code_fences(code), "optimized.cpp")
    is_windows = platform.system() == "Windows"
    
    try:
        # Compile command with optimizations
        if is_windows:
            compile_cmd = ["g++", "-Ofast", "-std=c++17", "-o", "optimized.exe", "optimized.cpp"]
            run_cmd = ["optimized.exe"]
        else:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=native", "-o", "optimized", "optimized.cpp"]
            run_cmd = ["./optimized"]
            
        # Compile
        compile_result = subprocess.run(
            compile_cmd, check=True, text=True, capture_output=True, shell=is_windows
        )
        
        # Run
        start_time = time.time()
        run_result = subprocess.run(
            run_cmd, check=True, text=True, capture_output=True, shell=is_windows
        )
        end_time = time.time()
        
        # Add execution time if not included in output
        output = run_result.stdout
        if "Execution Time:" not in output:
            output += f"\nExecution Time: {(end_time - start_time):.6f} seconds"
        
        return output
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"


In [ ]:
def execute_java(code):
    """Compile and execute Java code dynamically."""
    #  Determine the class name from the code
    import re
    class_pattern = re.compile(r"public\s+class\s+(\w+)")
    match = class_pattern.search(code)
    
    if match:
        class_name = match.group(1)
    else:
        class_name = "Optimized"  # Default class name
        
    file_name = f"{class_name}.java"


    write_output(strip_code_fences(code), file_name)
    
    is_windows = platform.system() == "Windows"

    try:
        # Compile the Java code
        compile_cmd = ["javac", file_name]
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True, shell=is_windows)
        
        # Run the compiled Java program
        run_cmd = ["java", class_name]
        start_time = time.time()
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True, shell=is_windows)
        end_time = time.time()
        
        # Add execution time if not included in output
        output = run_result.stdout
        if "Execution Time:" not in output:
            output += f"\nExecution Time: {(end_time - start_time):.6f} seconds"
            
        return output
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"


In [10]:
def huggingface_stream(python, language="cpp", model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"):
    """Generate code using a HuggingFace model via their Inference API with proper authentication."""
    from huggingface_hub import InferenceClient
    import os

    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")

    # Get API key from environment variable or let user provide it
    api_token = os.environ.get("HF_TOKEN")
    if not api_token:
        print("Warning: HF_TOKEN environment variable not found. Some models may require authentication.")
        
    # Customize prompt for model
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    
    # Format prompt based on model type (different models expect different formats)
    if "mistral" in model_name.lower() or "mixtral" in model_name.lower():
        full_prompt = f"<s>[INST] {system_message}\n\n{user_message} [/INST]"
    elif "llama" in model_name.lower():
        full_prompt = f"<s>[INST] {system_message}\n\n{user_message} [/INST]"
    elif "starcoder" in model_name.lower():
        full_prompt = f"<system>\n{system_message}\n</system>\n\n<user>\n{user_message}\n</user>\n\n<assistant>"
    elif "gemma" in model_name.lower():
        full_prompt = f"<start_of_turn>system\n{system_message}<end_of_turn>\n<start_of_turn>user\n{user_message}<end_of_turn>\n<start_of_turn>model\n"
    else:
        full_prompt = f"System: {system_message}\n\nUser: {user_message}\n\nAssistant:"

    try:
        # Initialize the client with the API token if available
        client = InferenceClient(model_name, token=api_token)
        
        # Define model-specific parameters
        params = {
            "max_new_tokens": 2048,
            "stream": True,
            "details": False,
            "temperature": 0.2,
            "top_p": 0.95,
        }
        
        # Add stop sequences based on model type
        if "mistral" in model_name.lower() or "mixtral" in model_name.lower() or "llama" in model_name.lower():
            params["stop_sequences"] = ["</s>"]
        
        full_response = ""
        for response in client.text_generation(full_prompt, **params):
            full_response += response
            yield strip_code_fences(full_response)

    except Exception as e:
        yield f"Error using HuggingFace model ({model_name}): {str(e)}\n\nPlease check your internet connection or Hugging Face API access."

In [ ]:
def ollama_stream(python, language="cpp"):
    """Generate code using the Ollama API with streaming."""
    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")
    
    # Set up the prompt for code conversion
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    
    # Streaming response with Ollama API
    url = "http://localhost:11434/api/generate"
    payload = {# change model if needed here
        "model": "codellama",  # Using the codellama model from Ollama
        "prompt": f"{system_message}\n\n{user_message}",
        "stream": False  # Set to True for streaming, False for complete response
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        if payload["stream"]:
            # This would handle streaming if needed
            full_response = ""
            for line in response.iter_lines():
                if line:
                    chunk = response.json()
                    if "response" in chunk:
                        full_response += chunk["response"]
                        yield full_response
        else:
            # Return the full response
            result = response.json()
            yield result["response"]
            
    except requests.exceptions.RequestException as e:
        yield f"Error connecting to Ollama API: {str(e)}\n\nPlease make sure Ollama is running with the llama model."


In [ ]:
def deepseek_stream(python, language="cpp"):
    """
    Generate code using the DeepSeek model via Ollama API with improved prompting.
    
    Args:
        python (str): Python code to convert  
        language (str): Target language (cpp or java)
        
    Yields:
        str: Generated code with proper formatting
    """
    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")
    
    # Set up the prompt for code conversion with specific instructions to avoid thinking
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    
    # Enhanced system prompt to get clean code output
    enhanced_system = (
        f"{system_message}\n"
        "IMPORTANT: Respond ONLY with clean code. No explanations, no thinking sections, "
        f"just the complete {language.upper()} implementation. Do not use code fences or ```."
    )
    
    # API request
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "deepseek-r1", 
        "prompt": f"{enhanced_system}\n\n{user_message}",
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        if payload["stream"]:
            # This would handle streaming if needed
            full_response = ""
            for line in response.iter_lines():
                if line:
                    chunk = response.json()
                    if "response" in chunk:
                        full_response += chunk["response"]
                        yield strip_code_fences(full_response)
        else:
            # Return the full response with fences and thinking removed
            result = response.json()
            yield strip_code_fences(result["response"])
            
    except requests.exceptions.RequestException as e:
        yield f"Error connecting to Ollama API: {str(e)}\n\nPlease make sure Ollama is running with the DeepSeek model."


In [13]:
def mixtral_stream(python, language="cpp"):
    """Generate code using Mixtral model via Hugging Face."""
    from huggingface_hub import InferenceClient

    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")

    # Customize prompt for Mixtral
    system_message = system_message_for(language)
    user_message = user_prompt_for(python)
    full_prompt = f"<s>[INST] {system_message}\n\n{user_message} [/INST]"

    try:
        client = InferenceClient("mistralai/Mixtral-8x7B-Instruct-v0.1")
        full_response = ""
        for response in client.text_generation(
            full_prompt,
            max_new_tokens=2048,
            stream=True,
            details=False,
            typical_p=0.7,
            temperature=0.3,
            stop_sequences=["</s>"]
        ):
            full_response += response
            yield full_response

    except Exception as e:
        yield f"Error using Mixtral model: {str(e)}\n\nPlease check your internet connection or Hugging Face API access."


In [14]:
def optimize(python, model="Ollama", language="cpp", api_key=None):
    """
    Optimize the given Python code using the specified model and generate the output
    in the requested programming language.
    
    Args:
        python (str): Python code to convert
        model (str): Model identifier to use
        language (str): Target language (cpp or java)
        api_key (str): HuggingFace API key (only for HF models)
        
    Yields:
        str: Generated code with proper formatting
    """
    # Map friendly model names to actual model paths for HuggingFace
    model_paths = {
        "CodeLlama-7B": "codellama/CodeLlama-7b-hf",
        "StarCoder-1B": "bigcode/starcoderbase-1b", 
        "StarCoder-3B": "bigcode/starcoderbase-3b",
        "CodeGemma-2B": "google/codegemma-2b",
        "Gemma-2B": "google/gemma-2b",
        "TinyLlama-1.1B": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    }
    
    if model == "Ollama":
        stream = ollama_stream(python, language=language)
    elif model == "DeepSeek":
        stream = deepseek_stream(python, language=language)
    elif model == "Mixtral-8x7B":
        stream = huggingface_stream(python, language=language, 
                                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1")
    else:
        raise ValueError("Unknown model. Please choose a supported model.")
    for stream_so_far in stream:
        yield stream_so_far

In [15]:
css = """
:root {
    --python-color: #3572A5;
    --cpp-color: #004482;
    --java-color: #B07219;
    --background-color: #F7F8FA;
    --text-color: #1A202C;
    --card-bg: #FFFFFF;
    --border-color: #E2E8F0;
    --shadow: 0 4px 6px -1px rgba(0,0,0,0.1), 0 2px 4px -1px rgba(0,0,0,0.06);
    --primary-font: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
}

body {
    background-color: var(--background-color);
    color: var(--text-color);
    margin: 0;
}

.gradio-container {
    font-family: var(--primary-font);
    max-width: 1200px;
    margin: 0 auto;
    padding: 2rem;
}

.app-header {
    text-align: center;
    margin-bottom: 2.5rem;
    padding: 2rem;
    background: linear-gradient(135deg, var(--python-color), var(--cpp-color));
    border-radius: 12px;
    color: white;
    box-shadow: var(--shadow);
}

.app-header h1 {
    margin: 0;
    font-size: 2rem;
    font-weight: 700;
}

.app-header p {
    margin: 0.5rem 0 0;
    opacity: 0.9;
}

.card {
    background: var(--card-bg);
    border-radius: 12px;
    box-shadow: var(--shadow);
    padding: 1.5rem;
    margin-bottom: 1.5rem;
    border: 1px solid var(--border-color);
}

.code-box {
    border-radius: 8px;
    border: 1px solid var(--border-color);
    font-family: 'JetBrains Mono', monospace;
    position: relative;
    resize: vertical;
    min-height: 250px;
    max-height: 500px;
    font-size: 0.9rem;
    transition: border-color 0.2s ease;
}

.code-box:focus-within {
    border-color: var(--python-color);
}

.python-box { border-color: var(--python-color); }
.cpp-box { border-color: var(--cpp-color); }
.java-box { border-color: var(--java-color); }

.output-box {
    background-color: var(--card-bg);
    border-radius: 8px;
    border: 1px solid var(--border-color);
    min-height: 100px;
    font-family: 'JetBrains Mono', monospace;
    font-size: 0.9rem;
}

.python-result { border-color: var(--python-color); }
.cpp-result { border-color: var(--cpp-color); }
.java-result { border-color: var(--java-color); }

.btn {
    border-radius: 8px;
    font-weight: 600;
    padding: 0.75rem 1.5rem;
    border: none;
    cursor: pointer;
    transition: all 0.2s ease;
    font-family: var(--primary-font);
}

.btn:disabled {
    opacity: 0.6;
    cursor: not-allowed;
}

.python-btn { background-color: var(--python-color); color: white; }
.cpp-btn { background-color: var(--cpp-color); color: white; }
.java-btn { background-color: var(--java-color); color: white; }
.compare-btn { 
    background-color: #2D3748; 
    color: white; 
}

.btn:hover:not(:disabled) {
    transform: translateY(-1px);
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    filter: brightness(1.05);
}

.comparison-table {
    width: 100%;
    border-collapse: separate;
    border-spacing: 0;
    background: #F0F4F8;
    border-radius: 8px;
    overflow: hidden;
    box-shadow: var(--shadow);
}

.comparison-table th, .comparison-table td {
    border: none;
    padding: 1rem;
    color: #1A202C;
    text-align: center;
}

.comparison-table th {
    background-color: #CBD5E0; /* Slightly darker gray for header */
    color: #1A202C;
    font-weight: 600;
    text-transform: uppercase;
    font-size: 0.85rem;
}

.comparison-table tr:nth-child(even) {
    background-color: #E2E8F0; 
}

.comparison-table tr:nth-child(odd) {
    background-color: #F0F4F8;
}


.copy-btn {
    position: absolute;
    top: 10px;
    right: 10px;
    background: var(--card-bg);
    border: 1px solid var(--border-color);
    border-radius: 6px;
    padding: 0.4rem 0.8rem;
    cursor: pointer;
    font-size: 0.8rem;
    font-family: var(--primary-font);
    transition: all 0.2s ease;
}

.copy-btn:hover {
    background: #EDF2F7;
}

.dropdown-container {
    display: flex;
    gap: 1rem;
    margin-bottom: 1.5rem;
}

@media (max-width: 768px) {
    .gradio-container {
        padding: 1rem;
    }
    
    .app-header {
        padding: 1.5rem;
    }
    
    .card {
        padding: 1rem;
    }
    
    .btn {
        padding: 0.6rem 1rem;
    }
    
    .dropdown-container {
        flex-direction: column;
        gap: 0.5rem;
    }
}
"""

In [16]:
def create_ui():
    with gr.Blocks(css=css, theme=gr.themes.Soft()) as ui:
        state = gr.State({"is_running": False, "last_copy_id": None})
        
        gr.HTML("""
        <div class="app-header">
            <h1>Code Conversion & Performance Suite</h1>
            <p>Professional tool for code transformation and runtime analysis</p>
        </div>
        """)
        
        with gr.Row(elem_classes="dropdown-container"):
            example_dropdown = gr.Dropdown(
                choices=list(EXAMPLES.keys()),
                label="Example Code",
                value="Pi Calculation",
                interactive=True,
                scale=1
            )
            model_dropdown = gr.Dropdown(
                choices=["Ollama", "DeepSeek", "Mixtral-8x7B"],
                label="AI Model",
                value="Ollama",
                interactive=True,
                scale=1
            )
        
        with gr.Row():
            with gr.Column(elem_classes="card"):
                python_code = gr.Code(
                    label="Python Source Code",
                    language="python",
                    value=EXAMPLES["Pi Calculation"],
                    elem_classes=["code-box", "python-box"],
                    lines=15
                )
                with gr.Row():
                    python_run_btn = gr.Button(
                        "Execute Python", 
                        elem_classes=["btn", "python-btn"],
                        elem_id="run-python"
                    )
                    convert_cpp_btn = gr.Button(
                        "Convert to C++", 
                        elem_classes=["btn", "cpp-btn"],
                        elem_id="convert-cpp"
                    )
                    convert_java_btn = gr.Button(
                        "Convert to Java", 
                        elem_classes=["btn", "java-btn"],
                        elem_id="convert-java"
                    )
                python_output = gr.TextArea(
                    label="Python Output",
                    elem_classes=["output-box", "python-result"],
                    lines=5
                )
        
        with gr.Row():
            with gr.Column(elem_classes="card"):
                cpp_code = gr.Code(
                    label="C++ Code",
                    language="cpp",
                    elem_classes=["code-box", "cpp-box"],
                    lines=15
                )
                cpp_run_btn = gr.Button(
                    "Execute C++", 
                    elem_classes=["btn", "cpp-btn"],
                    elem_id="run-cpp"
                )
                cpp_output = gr.TextArea(
                    label="C++ Output",
                    elem_classes=["output-box", "cpp-result"],
                    lines=5
                )
            
            with gr.Column(elem_classes="card"):
                java_code = gr.Code(
                    label="Java Code",
                    language="python",
                    elem_classes=["code-box", "java-box"],
                    lines=15
                )
                java_run_btn = gr.Button(
                    "Execute Java", 
                    elem_classes=["btn", "java-btn"],
                    elem_id="run-java"
                )
                java_output = gr.TextArea(
                    label="Java Output",
                    elem_classes=["output-box", "java-result"],
                    lines=5
                )
        
        with gr.Row(elem_classes="card"):
            compare_btn = gr.Button(
                "Analyze Performance", 
                elem_classes=["btn", "compare-btn"],
                elem_id="compare"
            )
            comparison_results = gr.HTML(
                label="Performance Analysis",
                elem_classes=["comparison-table"]
            )
        
        # JavaScript for copy functionality
        gr.HTML("""
        <script>
        async function copyCode(elementId) {
            const codeElement = document.querySelector(`#${elementId} textarea`);
            const text = codeElement.value;
            await navigator.clipboard.writeText(text);
            
            const btn = document.querySelector(`#${elementId} .copy-btn`);
            btn.textContent = 'Copied';
            setTimeout(() => btn.textContent = 'Copy', 2000);
        }
        
        document.addEventListener('DOMContentLoaded', () => {
            const codeBlocks = document.querySelectorAll('.code-box');
            codeBlocks.forEach(block => {
                const btn = document.createElement('button');
                btn.className = 'copy-btn';
                btn.textContent = 'Copy';
                btn.onclick = () => copyCode(block.closest('.gradio-container').id);
                block.parentElement.appendChild(btn);
            });
        });
        </script>
        """)
        
        # Event handlers
        def update_example(example, state):
            state["is_running"] = True
            try:
                return EXAMPLES[example], state
            except KeyError:
                return "Error: Invalid example selected", state
            finally:
                state["is_running"] = False
        
        def convert_code(code, model, target_lang, state):
            state["is_running"] = True
            try:
                if not code.strip():
                    return "Error: Empty code provided", state
                return next(optimize(code, model, target_lang)), state
            except Exception as e:
                return f"Error: {str(e)}", state
            finally:
                state["is_running"] = False
        
        def run_code(code, lang, state):
            state["is_running"] = True
            try:
                if lang == "python":
                    return execute_python(code), state
                elif lang == "cpp":
                    return execute_cpp(code), state
                else:
                    return execute_java(code), state
            except Exception as e:
                return f"Error: {str(e)}", state
            finally:
                state["is_running"] = False
        
        def compare_performance(python_code, cpp_code, java_code, 
                              python_output, cpp_output, java_output, state):
            state["is_running"] = True
            try:
                def extract_time(output):
                    time_pattern = re.compile(r"Execution Time: (\d+\.\d+) seconds")
                    match = time_pattern.search(output)
                    return float(match.group(1)) if match else None
                
                # Execute if outputs are empty
                if not python_output and python_code.strip():
                    python_output = execute_python(python_code)
                if not cpp_output and cpp_code.strip():
                    cpp_output = execute_cpp(cpp_code)
                if not java_output and java_code.strip():
                    java_output = execute_java(java_code)
                
                # Extract times
                python_time = extract_time(python_output) if python_output else None
                cpp_time = extract_time(cpp_output) if cpp_output else None
                java_time = extract_time(java_output) if java_output else None
                
                # Create comparison table
                table = "<table class='comparison-table' style='color: var(--text-color);'>"
                table += "<tr><th>Language</th><th>Time (seconds)</th><th>Speedup</th></tr>"
                
                # Python row
                python_display = f"{python_time:.6f}" if python_time is not None else "N/A"
                table += f"<tr><td>Python</td><td>{python_display}</td><td>1.00x</td></tr>"
                
                # C++ row
                if cpp_time is not None:
                    speedup = python_time / cpp_time if python_time else 0
                    table += f"<tr><td>C++</td><td>{cpp_time:.6f}</td><td>{speedup:.2f}x</td></tr>"
                else:
                    table += "<tr><td>C++</td><td>N/A</td><td>N/A</td></tr>"
                
                # Java row
                if java_time is not None:
                    speedup = python_time / java_time if python_time else 0
                    table += f"<tr><td>Java</td><td>{java_time:.6f}</td><td>{speedup:.2f}x</td></tr>"
                else:
                    table += "<tr><td>Java</td><td>N/A</td><td>N/A</td></tr>"
                
                table += "</table>"
                
                # Performance summary
                times = [(k, v) for k, v in {
                    "Python": python_time,
                    "C++": cpp_time,
                    "Java": java_time
                }.items() if v is not None]
                if times:
                    fastest = min(times, key=lambda x: x[1])[0]
                    table += f"""
                    <div style='margin-top: 1rem; padding: 1rem; background: #EDF2F7; border-radius: 8px;'>
                        <p style='margin: 0; font-weight: 600;'>Fastest: {fastest} ({min([t for t in [python_time, cpp_time, java_time] if t is not None]):.6f}s)</p>
                    </div>
                    """
                
                return table, python_output, cpp_output, java_output, state
            except Exception as e:
                return f"Error: {str(e)}", python_output, cpp_output, java_output, state
            finally:
                state["is_running"] = False
        
        # Event bindings
        example_dropdown.change(
            fn=update_example,
            inputs=[example_dropdown, state],
            outputs=[python_code, state]
        )
        
        convert_cpp_btn.click(
            fn=convert_code,
            inputs=[python_code, model_dropdown, gr.State("cpp"), state],
            outputs=[cpp_code, state]
        )
        
        convert_java_btn.click(
            fn=convert_code,
            inputs=[python_code, model_dropdown, gr.State("java"), state],
            outputs=[java_code, state]
        )
        
        python_run_btn.click(
            fn=run_code,
            inputs=[python_code, gr.State("python"), state],
            outputs=[python_output, state]
        )
        
        cpp_run_btn.click(
            fn=run_code,
            inputs=[cpp_code, gr.State("cpp"), state],
            outputs=[cpp_output, state]
        )
        
        java_run_btn.click(
            fn=run_code,
            inputs=[java_code, gr.State("java"), state],
            outputs=[java_output, state]
        )
        
        compare_btn.click(
            fn=compare_performance,
            inputs=[python_code, cpp_code, java_code, 
                   python_output, cpp_output, java_output, state],
            outputs=[comparison_results, python_output, cpp_output, java_output, state]
        )
        
        return ui

In [17]:
import io
import sys

def execute_python(code):
    """
    Execute Python code dynamically and capture its output.

    Args:
        code (str): The Python code to execute.

    Returns:
        str: The captured standard output of the executed code.

    Raises:
        Exception: If the execution of the code raises an error.
    """
    output = io.StringIO()
    try:
        sys.stdout = output  # Redirect standard output to the StringIO object
        exec(code, {})  # Execute code with an empty global context for safety
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        sys.stdout = sys.__stdout__  # Restore standard output

    return output.getvalue()


In [18]:
# Main function to start the app
def main():
    ui = create_ui()
    ui.launch(debug=True)

if __name__ == "__main__":
    main()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
